# Simple ipywidgets interface to Demo with Voila

In [2]:
import os
import warnings
from pathlib import Path

from ipywidgets import interact, fixed
import pandas as pd
import plotly.express as px

warnings.simplefilter(action='ignore', category=FutureWarning)
data_path = Path(os.getenv("MELBVIZ_DATA_PATH", "../data"))
counts_csv_path = data_path / "Pedestrian_Counting_System___2009_to_Present__counts_per_hour_.csv"
counts_df = pd.read_csv(counts_csv_path, index_col="ID", parse_dates=["Date_Time"]);

In [3]:
def plot_months(counts_df, year, truncate_y=False):
    """Plot Monthly traffic for a given year."""
    
    # 1. Collect and shape data
    year_df = counts_df[counts_df["Year"] == year]
    months_df = year_df.groupby("Month")["Hourly_Counts"].sum().reset_index(name="total_counts")
    sorted_months_df = months_df.sort_values(
        by="Month", 
        key=lambda x:pd.to_datetime(x, format="%B").dt.month
    )

    # 2. Make plot
    figure = px.line(
        sorted_months_df, 
        x="Month", 
        y="total_counts", 
        title="Monthly Total Pedestrian Counts",
        width=1500,
        height=500
    )
    
    # 3. Fine-tune plot's appearance 
    figure.update_traces(mode='lines+markers')
    figure.update_layout(yaxis_title="Total Pedestrian Counts", title_x=0.5)
    if not truncate_y:
        figure.update_layout(yaxis_rangemode='tozero')
    
    return figure

In [4]:
interact(plot_months, year=range(2009, 2021), counts_df=fixed(counts_df));

interactive(children=(Dropdown(description='year', options=(2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 20…